# Transformer
*Transformer network with Multi-Head Attention*

In [1]:
from chatbot.transformer import Transformer
import pandas as pd

In [2]:
df = pd.read_csv('../data/translation/dataset.csv', sep=';')
tf = Transformer(lang='en-us', num_epoch=2)

In [3]:
tf.fit(data=df)

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV2'>: Dictionary generation Corpus(lang='en-us', max_length=40, batch_size=64, buffer_size=20000, _start_token=0, _end_token=0, _vocab_size=0). 
	Please be patient. Depending on the size of the dataset and 
	the number of unique words, this can take a while (up to 5 minutes).
Data loaded with hyperparams: Corpus(lang='en-us', max_length=40, batch_size=64, buffer_size=20000, _start_token=[74785], _end_token=[74786], _vocab_size=74787).
<keras.engine.functional.Functional object at 0x00000264B62CD5A0> compiled successfully.
Epoch 1/2
692/692 [==============================] - 226s 304ms/step - loss: 2.8436 - _count_accuracy: 0.0581
Epoch 2/2
692/692 [==============================] - 211s 305ms/step - loss: 1.9010 - _count_accuracy: 0.1108


# DNN

*Bag-Of-Words based DNN network*

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import json
import pickle
import random

nltk.download("punkt")

#Loading intents.json
with open('intents.json') as intents:
  data = json.load(intents)

stemmer = LancasterStemmer()

try:
    with open('data.pickle','rb') as f:
        words, labels, training, output = pickle.load(f)
except:
# Fetching and Feeding information--
    words = []
    labels = []
    x_docs = []
    y_docs = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            x_docs.append(wrds)
            y_docs.append(intent['tag'])

            if intent['tag'] not in labels:
                labels.append(intent['tag'])

    words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    # One hot encoding, Converting the words to numerals
    for x, doc in enumerate(x_docs):
        bag = []
        wrds = [stemmer.stem(w) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)


        output_row = out_empty[:]
        output_row[labels.index(y_docs[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = np.array(training)
    output = np.array(output)

    with open('data.pickle','wb') as f:
        pickle.dump((words, labels, training, output), f)


net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:

    model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
    model.save('model.tflearn')


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


def chat():
    print("The bot is ready to talk!!(Type 'quit' to exit)")
    while True:
        inp = input("\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]


        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
            print("Bot:" + random.choice(responses))


chat()

# Default Sequential model